# L5: Content Creation at Scale

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

## Initial Imports

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Creating Structured Output

In [2]:
from pydantic import BaseModel, Field
from typing import List

class SocialMediaPost(BaseModel):
    platform: str = Field(..., description="The social media platform where the post will be published (e.g., Twitter, LinkedIn).")
    content: str = Field(..., description="The content of the social media post, including any hashtags or mentions.")

class ContentOutput(BaseModel):
    article: str = Field(..., description="The article, formatted in markdown.")
    social_media_posts: List[SocialMediaPost] = Field(..., description="A list of social media posts related to the article.")

## Loading Tasks and Agents YAML files

In [3]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Importing CrewAI Tools

In [4]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

## Setup Multi LLM models

In [5]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'
groq_llm = "groq/llama-3.1-70b-versatile"

## Creating Crew, Agents, and Tasks

In [6]:
# Creating Agents
market_news_monitor_agent = Agent(
    config=agents_config['market_news_monitor_agent'],
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    llm=groq_llm,
)

data_analyst_agent = Agent(
    config=agents_config['data_analyst_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
    llm=groq_llm,
)

content_creator_agent = Agent(
    config=agents_config['content_creator_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
)

quality_assurance_agent = Agent(
    config=agents_config['quality_assurance_agent'],
)

# Creating Tasks
monitor_financial_news_task = Task(
    config=tasks_config['monitor_financial_news'],
    agent=market_news_monitor_agent
)

analyze_market_data_task = Task(
    config=tasks_config['analyze_market_data'],
    agent=data_analyst_agent
)

create_content_task = Task(
    config=tasks_config['create_content'],
    agent=content_creator_agent,
    context=[monitor_financial_news_task, analyze_market_data_task]
)

quality_assurance_task = Task(
    config=tasks_config['quality_assurance'],
    agent=quality_assurance_agent,
    output_pydantic=ContentOutput
)

# Creating Crew
content_creation_crew = Crew(
    agents=[
        market_news_monitor_agent,
        data_analyst_agent,
        content_creator_agent,
        quality_assurance_agent
    ],
    tasks=[
        monitor_financial_news_task,
        analyze_market_data_task,
        create_content_task,
        quality_assurance_task
    ],
    verbose=True
)

## Kicking off the Crew

In [7]:
result = content_creation_crew.kickoff(inputs={
  'subject': 'Inflation in the US and the impact on the stock market in 2024'
})

# Agent: Lead Market Analyst
## Task: Monitor and analyze the latest news and updates related to the financial markets, with a particular focus on Inflation in the US and the impact on the stock market in 2024. Identify and summarize the most relevant and impactful news items that could influence market trends or investor decisions. Utilize financial news APIs and real-time market data tools to gather up-to-date information. Focus on detecting trends, regulatory changes, or significant economic indicators that directly relate to Inflation in the US and the impact on the stock market in 2024.



# Agent: Lead Market Analyst
## Thought: Thought: To gather the most relevant and up-to-date information on Inflation in the US and its impact on the stock market in 2024, I should search the internet for recent news articles and updates from reputable financial sources.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"US inflation news 2024 and stock market impact\"}"
##



# Agent: Lead Market Analyst
## Thought: Thought: I have obtained a detailed article on the impact of the recent US presidential election on the stock market. However, I still need to collect and analyze more data points and related news to generate a comprehensive report. To further assess the effect of inflation on the US stock market, I should analyze more recent financial news articles.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://www.wsj.com/livecoverage/stock-market-cpi-inflation-dow-sp500-nasdaq-live-10-10-2024\"}"
## Tool Output: 
wsj.comPlease enable JS and disable any ad blocker


You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:

Tool Name: Search the internet(**kwargs: Any) -> Any
Tool Description: Search the internet(search_query: 'string') - A tool that can be used to search the internet with a search_query. search_query: 'Mandatory search query you want to use to search the internet



# Agent: Chief Data Strategist
## Thought: Thought: To gather more data and insights about the impact of inflation in the US on the stock market in 2024, I should start by searching the internet for the latest news and trends related to this topic.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"US inflation news 2024 stock market impact\"}"
## Tool Output: 

Search results: Title: Stocks close higher after Fed cuts interest rates - Reuters
Link: https://www.reuters.com/markets/us/futures-steady-after-trump-fueled-rally-feds-rate-decision-awaited-2024-11-07/
Snippet: US weekly jobless claims rise moderately · Fed cuts rates by 25 bps · JP Morgan, Goldman weigh on Dow · Indexes: Dow flat, S&P 500 up 0.74%, Nasdaq ...
---
Title: Stock Market News, Oct. 10, 2024: Dow Slips After CPI Inflation ...
Link: https://www.wsj.com/livecoverage/stock-market-cpi-inflation-dow-sp500-nasdaq-live-10-10-2024
Snippet: US stocks declined Thursday after a hotter-than-expected inf



# Agent: Chief Data Strategist
## Thought: Thought: I now have access to various search results related to stock market performance in response to inflation in 2024. However, I need to analyze these results further to determine the impact of inflation on the stock market.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"impact of inflation on stock market performance 2024\"}"
## Tool Output: 

Search results: Title: Inflation Artificially Pumps Up the Stock Market
Link: https://www.heritage.org/budget-and-spending/commentary/inflation-artificially-pumps-the-stock-market
Snippet: Inflation can distort how we perceive market gains. While it may appear that investments in the stock market are yielding record-breaking ...
---
Title: How Inflation Affects The Stock Market | Bankrate
Link: https://www.bankrate.com/investing/how-inflation-affects-the-stock-market/
Snippet: Inflation impacts stock prices and sector performance broadly, too, which causes investors to r



# Agent: Creative Content Director
## Thought: I need to gather the latest insights and trends regarding inflation and its impact on the stock market in 2024 to create engaging content. Since I already have a detailed summary, I’ll start by searching for current developments or new insights on inflation in the US to enhance the content quality.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"US inflation trends October 2024 stock market impact\"}"
## Tool Output: 

Search results: Title: Stock Market News, Oct. 10, 2024: Dow Slips After CPI Inflation ...
Link: https://www.wsj.com/livecoverage/stock-market-cpi-inflation-dow-sp500-nasdaq-live-10-10-2024
Snippet: Annual consumer-price inflation slows less than expected, to 2.4% ... U.S. stocks declined Thursday after a hotter-than-expected inflation report ...
---
Title: US Consumer Confidence Bounced Back in October
Link: https://www.conference-board.org/topics/consumer-confidence/press/CCI-Oct-2024
Snippet: Co

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  2.26it/s]




# Agent: Creative Content Director
## Thought: Thought: I will continue to gather insights from additional reputable sources on inflation trends and their impact on the stock market to ensure the content I create is comprehensive and informative.
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"US inflation trends October 2024 stock market impact\", \"website\": \"https://www.cnbc.com\"}"
## Tool Output: 
Relevant Content:
outlook. In this scenario, continued geopolitical conflicts and trade actions cause inflation to stay higher for longer, driving the Federal Reserve to hike rates.1 Scenarios Baseline (70%): Real GDP growth slowed overall in the first quarter of 2024, coming in very close to our previous forecast’s expectations.2 We expect GDP growth will continue to moderate through the second half of this year and the start of the next, but the story is still positive overall. Consumer spending is forecasted to rise 2.3% this year, up from the 2.2



# Agent: Chief Content Officer
## Final Answer: 
```markdown
# Inflation in the US and Its Impact on the Stock Market in 2024

As we navigate through 2024, understanding the dynamics of inflation and its consequential impact on the stock market has never been more crucial for investors. This year has presented a complex landscape characterized by fluctuating inflation rates, changing consumer sentiments, and evolving market strategies. Here, we summarize the latest insights on inflation and its implications for stock market performance.

---

## Recent Developments in US Inflation

- **Inflation Rates**: According to the Consumer Price Index (CPI), inflation rose *2.4%* over the past year as of September 2024. Although this represents a slight decrease from the previous month, it remains above the Federal Reserve's targeted *2%* threshold, indicating persistent inflationary pressures. Many economists suggest that this elevated rate may influence the Federal Reserve's decision-making 

## Social Content

In [8]:
import textwrap

posts = result.pydantic.dict()['social_media_posts']
for post in posts:
    platform = post['platform']
    content = post['content']
    print(platform)
    wrapped_content = textwrap.fill(content, width=50)
    print(wrapped_content)
    print('-' * 50)

Twitter
📈💹 Inflation in the US has risen to 2.4% in 2024.
What does that mean for the stock market? Check
out our latest insights on how inflation is
impacting your investments! #Investing
#StockMarket #Inflation
--------------------------------------------------
LinkedIn
As inflation reaches 2.4% in 2024, investors need
to be proactive! Explore how changing consumer
sentiments and market reactions can guide your
investment strategies. #FinancialAdvisory
#InvestSmart
--------------------------------------------------
Facebook
The interplay between inflation and the stock
market in 2024 is crucial for investors! Discover
how to navigate these complexities and make
informed decisions. #InvestingWisely
#EconomicInsights
--------------------------------------------------


## Blog Post

In [9]:
from IPython.display import display, Markdown
display(Markdown(result.pydantic.dict()['article']))

# Inflation in the US and Its Impact on the Stock Market in 2024

As we navigate through 2024, understanding the dynamics of inflation and its consequential impact on the stock market has never been more crucial for investors. This year has presented a complex landscape characterized by fluctuating inflation rates, changing consumer sentiments, and evolving market strategies. Here, we summarize the latest insights on inflation and its implications for stock market performance.

---

## Recent Developments in US Inflation

- **Inflation Rates**: According to the Consumer Price Index (CPI), inflation rose *2.4%* over the past year as of September 2024. Although this represents a slight decrease from the previous month, it remains above the Federal Reserve's targeted *2%* threshold, indicating persistent inflationary pressures. Many economists suggest that this elevated rate may influence the Federal Reserve's decision-making and strategies moving forward.

- **Consumer Confidence**: Despite concerns over inflation, consumer confidence has rebounded. A recent report indicates that *51.4%* of consumers expect stock prices to increase over the next year, signaling optimism among investors even in the face of economic challenges. This rebound in consumer sentiment may also serve as a catalyst for spending, which is a critical driver of economic growth.

- **Market Reactions**: On October 10, 2024, major indices including the Dow Jones Industrial Average and S&P 500 saw declines following the release of a higher-than-expected inflation report. This suggests that market participants are closely monitoring inflation data as a key indicator of future economic stability. Investors are likely reassessing their exposure to equities in response to these macroeconomic signals.

---

## Analysis of Inflation's Impact on Stock Market Performance

- **Investor Sentiment**: The increasing inflation rates have generated caution among investors, with many waiting for indications of potential interest rate cuts from the Federal Reserve. A mixed economic outlook, paired with ongoing inflation, creates a challenging environment for market growth. Investors are advised to stay vigilant and adaptable in their strategies as the economic landscape evolves.

- **Sector Performance**: Inflation can distort perceptions of market gains. For instance, nominal increases in stock market indices since 2021 reflect a robust growth of *39%*, but when adjusted for inflation, these gains dwindle to only *15%*. Sectors like technology have achieved new highs; however, experts warn that inevitable corrections may be on the horizon as inflation continues to influence performance. In contrast, value stocks and those in essential services tend to hold more stable positions during turbulent economic cycles.

- **Economic Outlook**: Leading economic forecasts suggest that while the core inflation rate may stabilize at around *2.5%* by the end of 2024, other factors such as geopolitical tensions and supply chain disruptions could elongate high inflation periods. Thus, maintaining a diversified investment strategy remains essential for navigating this volatility. Investors should be prepared for continued fluctuations and potentially position their portfolios accordingly.

---

## Recommendations for Investors

1. **Stay Informed**: Regularly review inflation reports and understand their implications on monetary policy and market trends. Knowledge is power, and staying ahead of economic indicators can make a significant difference in investment outcomes.

2. **Diversify Portfolios**: Consider allocating investments across varied sectors to mitigate risks associated with inflation-driven market fluctuations. Diversification can act as a buffer against sector-specific downturns, providing a more stable investment portfolio.

3. **Adapt Strategies**: In response to rising prices, focus on growth sectors and defensive stocks that historically perform well during inflationary periods. This proactive approach can help investors navigate uncertain economic climates and seize potential opportunities.

4. **Utilize Multimedia Tools**: Leverage infographics and data visualizations to track trends and present analytical insights on inflation and market responses compellingly. Effective use of multimedia can enhance understanding and engagement with complex data.

---

## Conclusion

The interplay between inflation and the stock market in 2024 underscores a pivotal moment for investors. By comprehensively understanding inflation trends and proactively adapting to changing market conditions, investors can position themselves to capitalize on potential opportunities while safeguarding against inherent risks. Engaging content that educates and informs the audience on these developments is crucial for navigating the financial landscape in this era of uncertainty. 

The insights presented throughout this content aim to empower you in making informed investment decisions as we advance through this financial cycle.